## Installing Packages

In [14]:
# %pip install numpy
# %pip install pandas
# %pip install geojson
# %pip install shapely
# %pip install geopandas
# %pip install beautifulsoup4

## Importing Packages

In [15]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, MultiPoint, Polygon
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import numpy as np
import geojson
from shapely.wkt import loads
from shapely.geometry import mapping
from shapely.ops import unary_union

## Loading the geojson file

The challenging part of building the map is that there is no postal district geojson file which is publicly available. This means that the polygons for the districts will need to be custom drawn on the singapore map and exported as a geojson. I have tried this method but I found it extremely time consuming and not being a professional, the polygons drawn were not very accurate and high quality. 

Hence, one of the alternative approaches I had to consider was using the existing geojson maps available (there are geojson on planning areas) and then map it to the postal district. However, this proved to be challenging as well since certain districts were not covered in the planning areas like little india, macpherson/bradell, and more. As such I had to make use of a more detailed masterplan2019. 

We made use of the masterplan2019 with the subzone boundaries instead of the planning areas because it was more comprehensive and gave us a chance to cover all the districts unlike in planning area which missed out districts such as little india, macpherson/bradell, and more. Use subzone boundaries also allowed us to differentiate between those regions which belonged to multiple districts. 

I made use of a lookup table to map the planning areas to the districts based on the planning area names and subzone names as well but there were a lot which I was unable to map, for these I had to manually add them in by looking at the postal code and mapping it with the postal district. 

In [65]:
file_path = './MasterPlan2019SubzoneBoundaryNoSeaGEOJSON.geojson'
singapore_map = gpd.read_file(file_path)

properties = singapore_map.columns
properties

Index(['Name', 'Description', 'geometry'], dtype='object')

In [66]:
postal_df = pd.read_json("./singapore_postals.json")
print(postal_df.iloc[1].postals)

postal_mapping = []
for entry in postal_df['postals']:
    district = entry['district']
    location = entry['locations']
    for sector in entry['sectors']:
        postal_mapping.append({'postal_sector': sector, 'postal_district': district, 'locations': location})

postal_mapping_df = pd.DataFrame(postal_mapping)
postal_mapping_df.head()

{'id': '02', 'district': '02', 'sectors': ['07', '08'], 'locations': ['Anson', 'Tanjong Pagar']}


,postal_sector,postal_district,locations
0,01,01,"[Raffles Place, Cecil, Marina, People’s Park]"
1,02,01,"[Raffles Place, Cecil, Marina, People’s Park]"
2,03,01,"[Raffles Place, Cecil, Marina, People’s Park]"
3,04,01,"[Raffles Place, Cecil, Marina, People’s Park]"
4,05,01,"[Raffles Place, Cecil, Marina, People’s Park]"


In [67]:
singapore_map

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.81 1.2824 0, 103.82 1.2804 0, ..."
1,kml_2,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.82 1.2805 0, 103.82 1.2801 0, ..."
2,kml_3,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.84 1.2851 0, 103.84 1.2849 0, ..."
3,kml_4,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.85 1.2841 0, 103.85 1.2839 0, ..."
4,kml_5,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.85 1.2862 0, 103.85 1.2862 0, ..."
...,...,...,...
327,kml_328,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.84 1.3632 0, 103.84 1.3626 0, ..."
328,kml_329,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.84 1.3709 0, 103.84 1.3709 0, ..."
329,kml_330,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.85 1.3688 0, 103.85 1.3688 0, ..."
330,kml_331,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.85 1.3452 0, 103.85 1.345 0, 1..."


In [68]:
def parse_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    data_dict = {}
    if table:
        rows = table.find_all('tr')
        for row in rows[1:]:  # Skip the header row
            columns = row.find_all(['th', 'td'])
            if len(columns) == 2:
                key = columns[0].get_text(strip=True)
                value = columns[1].get_text(strip=True)
                data_dict[key] = value
    return data_dict


singapore_map['parsed_data'] = singapore_map['Description'].apply(parse_html)

# Expand the parsed data into individual columns
parsed_df = pd.json_normalize(singapore_map['parsed_data'])
singapore_map = pd.concat([singapore_map, parsed_df], axis=1)

# Drop the original HTML column if no longer needed
singapore_map = singapore_map.drop(columns=['Description', 'parsed_data'])
singapore_map.head()

,Name,geometry,SUBZONE_NO,SUBZONE_N,SUBZONE_C,CA_IND,PLN_AREA_N,PLN_AREA_C,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D
0,kml_1,"POLYGON Z ((103.81 1.2824 0, 103.82 1.2804 0, ...",12,DEPOT ROAD,BMSZ12,N,BUKIT MERAH,BM,CENTRAL REGION,CR,C22DED671DE2A940,20191223152313
1,kml_2,"POLYGON Z ((103.82 1.2805 0, 103.82 1.2801 0, ...",2,BUKIT MERAH,BMSZ02,N,BUKIT MERAH,BM,CENTRAL REGION,CR,085EF219A5A1AEAD,20191223152313
2,kml_3,"POLYGON Z ((103.84 1.2851 0, 103.84 1.2849 0, ...",3,CHINATOWN,OTSZ03,Y,OUTRAM,OT,CENTRAL REGION,CR,EF2B9A91AF49E025,20191223152313
3,kml_4,"POLYGON Z ((103.85 1.2841 0, 103.85 1.2839 0, ...",4,PHILLIP,DTSZ04,Y,DOWNTOWN CORE,DT,CENTRAL REGION,CR,615D4EDDEF809F8E,20191223152313
4,kml_5,"POLYGON Z ((103.85 1.2862 0, 103.85 1.2862 0, ...",5,RAFFLES PLACE,DTSZ05,Y,DOWNTOWN CORE,DT,CENTRAL REGION,CR,72107B11807074F4,20191223152313


In [71]:
singapore_map.SUBZONE_N.unique()

array(['DEPOT ROAD', 'BUKIT MERAH', 'CHINATOWN', 'PHILLIP',
       'RAFFLES PLACE', 'CHINA SQUARE', 'TIONG BAHRU', 'BAYFRONT SUBZONE',
       'TIONG BAHRU STATION', 'CLIFFORD PIER', 'MARINA SOUTH', 'NICOLL',
       'MARINA EAST', 'INSTITUTION HILL', 'ROBERTSON QUAY',
       'JURONG ISLAND AND BUKOM', 'FORT CANNING', "PEARL'S HILL",
       'BOAT QUAY', 'HENDERSON HILL', 'REDHILL', 'ALEXANDRA HILL',
       'BUKIT HO SWEE', 'CLARKE QUAY', 'PASIR PANJANG 1', 'QUEENSWAY',
       'MARINA EAST (MP)', 'SUDONG', 'SEMAKAU', 'SOUTHERN GROUP',
       'SENTOSA', 'CITY TERMINALS', 'ANSON', 'STRAITS VIEW',
       'MARITIME SQUARE', 'TELOK BLANGAH RISE', 'TANJONG PAGAR',
       'EVERTON PARK', 'TELOK BLANGAH WAY', 'MAXWELL', 'CECIL',
       'KAMPONG TIONG BAHRU', 'TELOK BLANGAH DRIVE', 'PASIR PANJANG 2',
       'CENTRAL SUBZONE', 'SINGAPORE GENERAL HOSPITAL', 'BUGIS',
       'VICTORIA', 'PATERSON', 'EAST COAST',
       "NATIONAL UNIVERSITY OF S'PORE", 'ONE TREE HILL', 'COMMONWEALTH',
       'DOVER', '

In [52]:
singapore_map.to_csv("sg_map_data.csv")

In [49]:
def find_postal_district(pln_area_n, subzone_n, mapping_df):
    for _, row in mapping_df.iterrows():
        # Check if PLN_AREA_N or SUBZONE_N matches any location in the mapping DataFrame
        if any(location.lower() in pln_area_n.lower() for location in row['locations']) or \
           any(location.lower() in subzone_n.lower() for location in row['locations']):
            return row['postal_district']
        # Debugging print statements
        print(f"PLN_AREA_N: {pln_area_n.lower()}, SUBZONE_N: {subzone_n.lower()}, Locations: {[location.lower() for location in row['locations']]}")
    return np.nan

In [50]:
singapore_map['postal_district'] = singapore_map.apply(
    lambda row: find_postal_district(row['PLN_AREA_N'], row['SUBZONE_N'], postal_mapping_df),
    axis=1
)

singapore_map.head()

PLN_AREA_N: bukit merah, SUBZONE_N: depot road, Locations: ['raffles place', 'cecil', 'marina', 'people’s park']
PLN_AREA_N: bukit merah, SUBZONE_N: depot road, Locations: ['raffles place', 'cecil', 'marina', 'people’s park']
PLN_AREA_N: bukit merah, SUBZONE_N: depot road, Locations: ['raffles place', 'cecil', 'marina', 'people’s park']
PLN_AREA_N: bukit merah, SUBZONE_N: depot road, Locations: ['raffles place', 'cecil', 'marina', 'people’s park']
PLN_AREA_N: bukit merah, SUBZONE_N: depot road, Locations: ['raffles place', 'cecil', 'marina', 'people’s park']
PLN_AREA_N: bukit merah, SUBZONE_N: depot road, Locations: ['anson', 'tanjong pagar']
PLN_AREA_N: bukit merah, SUBZONE_N: depot road, Locations: ['anson', 'tanjong pagar']
PLN_AREA_N: bukit merah, SUBZONE_N: depot road, Locations: ['queenstown', 'tiong bahru']
PLN_AREA_N: bukit merah, SUBZONE_N: depot road, Locations: ['queenstown', 'tiong bahru']
PLN_AREA_N: bukit merah, SUBZONE_N: depot road, Locations: ['queenstown', 'tiong bahr

,Name,geometry,SUBZONE_NO,SUBZONE_N,SUBZONE_C,CA_IND,PLN_AREA_N,PLN_AREA_C,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D,postal_district
0,kml_1,"POLYGON Z ((103.81 1.2824 0, 103.82 1.2804 0, ...",12,DEPOT ROAD,BMSZ12,N,BUKIT MERAH,BM,CENTRAL REGION,CR,C22DED671DE2A940,20191223152313,NaN
1,kml_2,"POLYGON Z ((103.82 1.2805 0, 103.82 1.2801 0, ...",2,BUKIT MERAH,BMSZ02,N,BUKIT MERAH,BM,CENTRAL REGION,CR,085EF219A5A1AEAD,20191223152313,NaN
2,kml_3,"POLYGON Z ((103.84 1.2851 0, 103.84 1.2849 0, ...",3,CHINATOWN,OTSZ03,Y,OUTRAM,OT,CENTRAL REGION,CR,EF2B9A91AF49E025,20191223152313,NaN
3,kml_4,"POLYGON Z ((103.85 1.2841 0, 103.85 1.2839 0, ...",4,PHILLIP,DTSZ04,Y,DOWNTOWN CORE,DT,CENTRAL REGION,CR,615D4EDDEF809F8E,20191223152313,NaN
4,kml_5,"POLYGON Z ((103.85 1.2862 0, 103.85 1.2862 0, ...",5,RAFFLES PLACE,DTSZ05,Y,DOWNTOWN CORE,DT,CENTRAL REGION,CR,72107B11807074F4,20191223152313,01


This is the result that I have achieved after performing the matching based on the lookup json table which I created after referencing from the URA website: 

https://www.ura.gov.sg/Corporate/-/media/Corporate/Property/PMI-Online/List_Of_Postal_Districts.pdf

In [51]:
singapore_map.postal_district.value_counts()

postal_district
22    35
12    19
03    18
27    17
20    14
10    12
18    11
25    10
19    10
09     9
16     8
24     7
17     6
11     6
01     6
14     5
28     5
04     3
02     2
13     1
08     1
15     1
26     1
Name: count, dtype: int64

In [20]:
csv_file = "sg_map_data.csv"
df = pd.read_csv(csv_file)
df = df[['geometry', 'SUBZONE_N', 'PLN_AREA_N', 'postal_district']]
df.head()

,geometry,SUBZONE_N,PLN_AREA_N,postal_district
0,POLYGON Z ((103.828545450459 1.45877524953475 ...,ADMIRALTY,SEMBAWANG,27.0
1,POLYGON Z ((103.901327898145 1.35658812288024 ...,AIRPORT ROAD,PAYA LEBAR,14.0
2,POLYGON Z ((103.81444653879 1.28547429883476 0...,ALEXANDRA HILL,BUKIT MERAH,NaN
3,POLYGON Z ((103.817404504805 1.29433192068692 ...,ALEXANDRA NORTH,BUKIT MERAH,NaN
4,POLYGON Z ((103.891267385312 1.32131770982548 ...,ALJUNIED,GEYLANG,14.0


In [21]:
filtered_df = df

# Handle NaN values in postal_district
filtered_df['postal_district'] = filtered_df['postal_district'].fillna(
    "Unknown")

# Group by postal_district and merge polygons
merged_features = []
for postal_district, group in filtered_df.groupby('postal_district'):
    try:
        # Combine all geometries for the postal_district
        geometries = [loads(geom) for geom in group['geometry']]
        merged_geometry = unary_union(geometries)  # Merge polygons

        # Get other properties from the first row in the group
        subzone_n = group['SUBZONE_N'].iloc[0]
        pln_area_n = group['PLN_AREA_N'].iloc[0]

        # Create GeoJSON feature
        feature = geojson.Feature(
            geometry=mapping(merged_geometry),
            properties={
                "postal_district": postal_district,
                "SUBZONE_N": subzone_n,
                "PLN_AREA_N": pln_area_n
            }
        )
        merged_features.append(feature)
    except Exception as e:
        print(f"Skipping postal_district {postal_district} due to geometry error: {e}")

# Create GeoJSON FeatureCollection
feature_collection = geojson.FeatureCollection(merged_features)

# Save to a GeoJSON file
output_file = "merged_output.geojson"
with open(output_file, "w") as f:
    geojson.dump(feature_collection, f)

print(f"Merged GeoJSON file created: {output_file}")

Merged GeoJSON file created: merged_output.geojson


## Converting to GEOJSON format

The final step is to take the csv file and convert to geojson since the d3 chloropleth map takes that data in that format. 

In [10]:
# Prepare GeoJSON features
features = []
for _, row in df.iterrows():
    # Convert WKT to GeoJSON geometry if necessary
    try:
        geometry = loads(row['geometry'])  # Assumes geometry is in WKT format
        geojson_geometry = mapping(geometry)
    except Exception as e:
        print(f"Skipping row due to geometry error: {e}")
        continue  # Skip rows with invalid geometry

    # Handle NaN values in postal_district
    postal_district = row['postal_district']
    if pd.isna(postal_district):  # Check if postal_district is NaN
        postal_district = "Unknown"  # Replace with a default value

    # Create GeoJSON feature
    feature = geojson.Feature(
        geometry=geojson_geometry,
        properties={
            "SUBZONE_N": row['SUBZONE_N'],
            "PLN_AREA_N": row['PLN_AREA_N'],
            "postal_district": postal_district
        }
    )
    features.append(feature)

# Create GeoJSON FeatureCollection
feature_collection = geojson.FeatureCollection(features)

# Save to a GeoJSON file
output_file = "output.geojson"
with open(output_file, "w") as f:
    geojson.dump(feature_collection, f)

print(f"GeoJSON file created: {output_file}")

GeoJSON file created: output.geojson
